<a href="https://colab.research.google.com/github/joaochenriques/IST_MCTE/blob/main/StreamTurbines/BladeDesign_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import minimize

In [4]:
def params( x, r ):

    ϕ = x[0]
    c = x[1]
    Z = 3
    σ = Z * c / ( 2 * np.pi * r )

    CL = 1.0
    CD = 0.02
    k = 1.0

    Ca = CL * np.cos(ϕ) + CD * np.sin(ϕ)
    Ct = CL * np.sin(ϕ) - CD * np.cos(ϕ)

    a  = ( σ * Ca ) / ( 4.0 * k * np.sin(ϕ)**2 + σ * Ca )
    ap = ( σ * Ct ) / ( 4.0 * k * np.sin(ϕ)*np.cos(ϕ) - σ * Ct )

    return k, a, ap

def objective( x, r ):
    k, a, ap = params( x, r )
    fO = k * ap * ( 1.0 - a ) * r**3

    return -fO

def constraint( x, r ):
    V1a = 2.25
    Ω = 15 * np.pi / 30.0
    k, a, ap = params( x, r )

    fC = ( 1.0 + ap ) * ap * ( Ω * r / V1a )**2 - ( 1 - a ) * a

    return fC

In [5]:
x0 = ( 0.1, 0.3 )

ϕ_bound = ( 0.0, np.pi/2.0 )
c_bound = ( 0.1, 2.0 )

r = 2.0
bnds = ( ϕ_bound, c_bound )
cons = [ { 'type': 'eq', 'fun': lambda x: constraint( x, r ) } ]

sol = minimize( objective, x0, args=(r,), method = 'SLSQP', bounds = bnds, constraints = cons )

print(sol)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0
       x: [ 0.000e+00  5.476e-01]
     nit: 2
     jac: [ 3.636e-06  0.000e+00]
    nfev: 6
    njev: 2
